In [101]:
import requests as reqs
from requests.packages.urllib3.exceptions import InsecureRequestWarning
reqs.packages.urllib3.disable_warnings(InsecureRequestWarning)
import pandas as pd

In [102]:
sc="https://www.meteo.lv/josso_security_check"
sec_check=reqs.post(sc)
cookies=sec_check.cookies.get_dict()
cookies["JSESSIONID"]

'C1813809A437971012467E949CE75D2F'

In [103]:
url="https://www.meteo.lv/meteorologija-datu-meklesana/?"

headers={
"Content-Type":"application/x-www-form-urlencoded",
"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36",
"Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
"Referer": "https://www.meteo.lv/meteorologija-datu-meklesana/?nid=461",
"Cookie": "JSESSIONID="+cookies["JSESSIONID"]
}

In [104]:
import json
saraksts=reqs.get("https://www.meteo.lv/klasifikatoru-filtrs/?iBy=station&iStation=&iParameter=4001&pMonitoringType=METEOROLOGY")
saraksts= json.loads(saraksts.text)

In [105]:
stacijuSaraksts={}
for each in saraksts["stations"][1:]:
    stacijuSaraksts[each["id"]]=each["name"]

parametruSaraksts={}
for each in saraksts["parameters"][1:]:
    parametruSaraksts[each["id"]]=each["name"]

In [106]:
parametruSaraksts

{'4515': 'Aramkārtas temperatūra 15 cm dziļumā, faktiskā',
 '4516': 'Aramkārtas temperatūra 20 cm dziļumā, faktiskā',
 '4513': 'Aramkārtas temperatūra 5 cm dziļumā, faktiskā',
 '4167': 'Atmosfēras spiediens stacijas līmenī, faktiskais',
 '4457': 'Augsnes virsmas stāvoklis',
 '4459': 'Augsnes virsmas temperatūra, faktiskā',
 '4464': 'Augsnes virsmas temperatūra, stundas maksimālā',
 '4462': 'Augsnes virsmas temperatūra, stundas minimālā',
 '4327': 'Augšējo mākoņu forma',
 '4001': 'Gaisa temperatūra, faktiskā',
 '4008': 'Gaisa temperatūra, maksimālā iepriekšējo 3 stundu laikā',
 '4003': 'Gaisa temperatūra, minimālā iepriekšējo 3 stundu laikā',
 '4006': 'Gaisa temperatūra, stundas maksimālā',
 '4004': 'Gaisa temperatūra, stundas minimālā',
 '4002': 'Gaisa temperatūra, stundas vidējā',
 '4321': 'Kopējais mākoņu daudzums',
 '10307': 'Laika apstakļi 1. kods pēdējā 1 stundā;A',
 '10308': 'Laika apstakļi 2. kods pēdējā 1 stundā;A',
 '10306': 'Laika apstakļi, faktiskie;A',
 '4627': 'Laika apstā

Vispirms savācam nokrišņus no Jelgavas, 2017-2021

In [110]:
stationID=30036 # 30094 - Rīga, 30036 - Jelgava
paramID=4570 # 4001 - gaisa temp vidējā, 4341 - sniega sega stundas vidējā, 4570 nokrišņu daudzums stundas summa
startYear=2017
endYear=2021

In [109]:
for year in range(startYear,endYear+1):
    StartDate="01.01."+str(year)
    EndDate="31.12."+str(year)
    params="iBy=station&nid=461&pMonitoringType=METEOROLOGY&iStation="+str(stationID)+"&iParameter="+str(paramID)+"&iDateFrom="+StartDate+"&iDateTill="+EndDate
    fname=stacijuSaraksts[str(stationID)]+"_" \
    +parametruSaraksts[str(paramID)] + "_" \
    +StartDate+"-" \
    +EndDate+".xls"
    print(fname)
    
    result=reqs.post(url,verify=False,data=params, headers=headers)
    print(url,params)
    with open(fname, 'wb') as f:
        f.write(result.content)

Jelgava_Nokrišņu daudzums, stundas summa_01.01.2017-31.12.2017.xls
https://www.meteo.lv/meteorologija-datu-meklesana/? iBy=station&nid=461&pMonitoringType=METEOROLOGY&iStation=30036&iParameter=4570&iDateFrom=01.01.2017&iDateTill=31.12.2017


In [111]:
df=[]

for enum, year in enumerate(range(startYear,endYear+1)):
    StartDate="01.01."+str(year)
    EndDate="31.12."+str(year)
    fname=stacijuSaraksts[str(stationID)]+"_" \
    +parametruSaraksts[str(paramID)] + "_" \
    +StartDate+"-" \
    +EndDate+".xls"
    #df[enum]=pd.read_excel(fname)
    df.append(pd.read_excel(fname,skiprows=1,parse_dates=["Datums \ Laiks"],index_col=0,dayfirst=True))
    #df[enum]["vidējā"]=df[enum].mean(axis=1)
    df[enum]["summa"]=df[enum].sum(axis=1)

In [112]:
nokrisni=pd.concat(df)
nokrisni=nokrisni[["summa"]]
nokrisni

,summa
Datums \ Laiks,
2017-01-01,0.2
2017-01-02,0.8
2017-01-03,0.0
2017-01-04,4.0
2017-01-05,0.2
...,...
2021-12-27,0.0
2021-12-28,0.0
2021-12-29,0.0


In [51]:
fname2=stacijuSaraksts[str(stationID)]+"_" \
    +parametruSaraksts[str(paramID)] + "_" \
    +str(startYear)+"-" \
    +str(endYear)+".xls"
dataf.to_excel(fname2)

tālāk savācam temperatūru no Jelgavas, 2017-2021

In [127]:
stationID=30036 # 30094 - Rīga, 30036 - Jelgava
paramID=4001 # 4001 - gaisa temp vidējā, 4341 - sniega sega stundas vidējā, 4570 nokrišņu daudzums stundas summa
startYear=2017
endYear=2021

In [55]:
for year in range(startYear,endYear+1):
    StartDate="01.01."+str(year)
    EndDate="31.12."+str(year)
    params="iBy=station&nid=461&pMonitoringType=METEOROLOGY&iStation="+str(stationID)+"&iParameter="+str(paramID)+"&iDateFrom="+StartDate+"&iDateTill="+EndDate
    fname=stacijuSaraksts[str(stationID)]+"_" \
    +parametruSaraksts[str(paramID)] + "_" \
    +StartDate+"-" \
    +EndDate+".xls"
    print(fname)
    
    result=reqs.post(url,verify=False,data=params, headers=headers)
    print(url,params)
    with open(fname, 'wb') as f:
        f.write(result.content)

Jelgava_Gaisa temperatūra, faktiskā_01.01.2017-31.12.2017.xls
https://www.meteo.lv/meteorologija-datu-meklesana/? iBy=station&nid=461&pMonitoringType=METEOROLOGY&iStation=30036&iParameter=4001&iDateFrom=01.01.2017&iDateTill=31.12.2017
Jelgava_Gaisa temperatūra, faktiskā_01.01.2018-31.12.2018.xls
https://www.meteo.lv/meteorologija-datu-meklesana/? iBy=station&nid=461&pMonitoringType=METEOROLOGY&iStation=30036&iParameter=4001&iDateFrom=01.01.2018&iDateTill=31.12.2018
Jelgava_Gaisa temperatūra, faktiskā_01.01.2019-31.12.2019.xls
https://www.meteo.lv/meteorologija-datu-meklesana/? iBy=station&nid=461&pMonitoringType=METEOROLOGY&iStation=30036&iParameter=4001&iDateFrom=01.01.2019&iDateTill=31.12.2019
Jelgava_Gaisa temperatūra, faktiskā_01.01.2020-31.12.2020.xls
https://www.meteo.lv/meteorologija-datu-meklesana/? iBy=station&nid=461&pMonitoringType=METEOROLOGY&iStation=30036&iParameter=4001&iDateFrom=01.01.2020&iDateTill=31.12.2020
Jelgava_Gaisa temperatūra, faktiskā_01.01.2021-31.12.2021.xl

In [128]:
df=[]

for enum, year in enumerate(range(startYear,endYear+1)):
    StartDate="01.01."+str(year)
    EndDate="31.12."+str(year)
    fname=stacijuSaraksts[str(stationID)]+"_" \
    +parametruSaraksts[str(paramID)] + "_" \
    +StartDate+"-" \
    +EndDate+".xls"
    #df[enum]=pd.read_excel(fname)
    df.append(pd.read_excel(fname,skiprows=1,parse_dates=["Datums \ Laiks"],index_col=0,dayfirst=True))
    df[enum]["dienas vidējā temperatūra, C"]=df[enum].mean(axis=1)
    #df[enum]["vidējā"]=df[enum].mean(axis=1)

In [67]:
df

[                00:00  01:00  02:00  03:00  04:00  05:00  06:00  07:00  08:00  \
 Datums \ Laiks                                                                  
 2017-01-01        6.3    6.5    6.5    6.5    6.4    6.4    6.3    6.2    6.2   
 2017-01-02        3.5    2.2    2.4    1.9    1.3    0.5    0.9    1.1    0.6   
 2017-01-03       -2.4   -1.6   -1.5   -2.2   -2.6   -2.7   -2.5   -2.2   -2.8   
 2017-01-04       -2.1   -1.6   -1.3   -2.0   -2.3   -2.4   -2.4   -2.3   -2.4   
 2017-01-05      -12.4  -12.9  -13.1  -13.4  -13.4  -13.1  -12.9  -13.1  -13.2   
 ...               ...    ...    ...    ...    ...    ...    ...    ...    ...   
 2017-12-27        3.1    2.9    2.6    2.3    2.4    1.5    1.5    1.3    1.2   
 2017-12-28        2.0    1.5    2.0    2.1    2.2    2.3    2.6    2.9    3.0   
 2017-12-29        3.2    3.1    3.3    3.3    3.2    3.0    3.0    3.0    2.7   
 2017-12-30        2.7    2.1    2.3    2.5    2.5    1.9    1.0    1.1    0.6   
 2017-12-31     

In [129]:
temperatura=pd.concat(df)
temperatura=temperatura[["dienas vidējā temperatūra, C"]]
temperatura

,"dienas vidējā temperatūra, C"
Datums \ Laiks,
2017-01-01,6.016667
2017-01-02,0.800000
2017-01-03,-2.641667
2017-01-04,-4.291667
2017-01-05,-13.229167
...,...
2021-12-27,-3.950000
2021-12-28,-8.379167
2021-12-29,-7.241667


savācam relatīvo mitrumu, Jelgava, 2017-2021

In [136]:
#4080
stationID=30036 # 30094 - Rīga, 30036 - Jelgava
paramID=4080 # 4001 - gaisa temp vidējā, 4341 - sniega sega stundas vidējā, 4570 nokrišņu daudzums stundas summa, 4080 -rel mitrums
startYear=2017
endYear=2021

In [132]:
for year in range(startYear,endYear+1):
    StartDate="01.01."+str(year)
    EndDate="31.12."+str(year)
    params="iBy=station&nid=461&pMonitoringType=METEOROLOGY&iStation="+str(stationID)+"&iParameter="+str(paramID)+"&iDateFrom="+StartDate+"&iDateTill="+EndDate
    fname=stacijuSaraksts[str(stationID)]+"_" \
    +parametruSaraksts[str(paramID)] + "_" \
    +StartDate+"-" \
    +EndDate+".xls"
    print(fname)
    
    result=reqs.post(url,verify=False,data=params, headers=headers)
    print(url,params)
    with open(fname, 'wb') as f:
        f.write(result.content)

Jelgava_Relatīvais mitrums, faktiskais_01.01.2017-31.12.2017.xls
https://www.meteo.lv/meteorologija-datu-meklesana/? iBy=station&nid=461&pMonitoringType=METEOROLOGY&iStation=30036&iParameter=4080&iDateFrom=01.01.2017&iDateTill=31.12.2017
Jelgava_Relatīvais mitrums, faktiskais_01.01.2018-31.12.2018.xls
https://www.meteo.lv/meteorologija-datu-meklesana/? iBy=station&nid=461&pMonitoringType=METEOROLOGY&iStation=30036&iParameter=4080&iDateFrom=01.01.2018&iDateTill=31.12.2018
Jelgava_Relatīvais mitrums, faktiskais_01.01.2019-31.12.2019.xls
https://www.meteo.lv/meteorologija-datu-meklesana/? iBy=station&nid=461&pMonitoringType=METEOROLOGY&iStation=30036&iParameter=4080&iDateFrom=01.01.2019&iDateTill=31.12.2019
Jelgava_Relatīvais mitrums, faktiskais_01.01.2020-31.12.2020.xls
https://www.meteo.lv/meteorologija-datu-meklesana/? iBy=station&nid=461&pMonitoringType=METEOROLOGY&iStation=30036&iParameter=4080&iDateFrom=01.01.2020&iDateTill=31.12.2020
Jelgava_Relatīvais mitrums, faktiskais_01.01.202

In [137]:
df=[]

for enum, year in enumerate(range(startYear,endYear+1)):
    StartDate="01.01."+str(year)
    EndDate="31.12."+str(year)
    fname=stacijuSaraksts[str(stationID)]+"_" \
    +parametruSaraksts[str(paramID)] + "_" \
    +StartDate+"-" \
    +EndDate+".xls"
    #df[enum]=pd.read_excel(fname)
    df.append(pd.read_excel(fname,skiprows=1,parse_dates=["Datums \ Laiks"],index_col=0,dayfirst=True))
    df[enum]["vidējais relatīvais mitrums"]=df[enum].mean(axis=1)
    #df[enum]["vidējā"]=df[enum].mean(axis=1)


'Jelgava_Relatīvais mitrums, faktiskais_01.01.2021-31.12.2021.xls'

In [140]:

relMitrums=pd.concat(df)
relMitrums=relMitrums[["vidējais relatīvais mitrums"]]
relMitrums

,vidējais relatīvais mitrums
Datums \ Laiks,
2017-01-01,94.000000
2017-01-02,95.500000
2017-01-03,89.583333
2017-01-04,88.958333
2017-01-05,68.000000
...,...
2021-12-27,89.083333
2021-12-28,90.000000
2021-12-29,72.000000


savācam sniega segas stundas biezumu, Jelgava, 2017-2021

In [120]:
stationID=30036 # 30094 - Rīga, 30036 - Jelgava
paramID=4341 # 4001 - gaisa temp vidējā, 4341 - sniega segas biezums stundas vidējā, cm, 4570 nokrišņu daudzums stundas summa
startYear=2017
endYear=2021

In [71]:
for year in range(startYear,endYear+1):
    StartDate="01.01."+str(year)
    EndDate="31.12."+str(year)
    params="iBy=station&nid=461&pMonitoringType=METEOROLOGY&iStation="+str(stationID)+"&iParameter="+str(paramID)+"&iDateFrom="+StartDate+"&iDateTill="+EndDate
    fname=stacijuSaraksts[str(stationID)]+"_" \
    +parametruSaraksts[str(paramID)] + "_" \
    +StartDate+"-" \
    +EndDate+".xls"
    print(fname)
    
    result=reqs.post(url,verify=False,data=params, headers=headers)
    print(url,params)
    with open(fname, 'wb') as f:
        f.write(result.content)

Jelgava_Sniega segas biezums, stundas vidējais_01.01.2017-31.12.2017.xls
https://www.meteo.lv/meteorologija-datu-meklesana/? iBy=station&nid=461&pMonitoringType=METEOROLOGY&iStation=30036&iParameter=4341&iDateFrom=01.01.2017&iDateTill=31.12.2017
Jelgava_Sniega segas biezums, stundas vidējais_01.01.2018-31.12.2018.xls
https://www.meteo.lv/meteorologija-datu-meklesana/? iBy=station&nid=461&pMonitoringType=METEOROLOGY&iStation=30036&iParameter=4341&iDateFrom=01.01.2018&iDateTill=31.12.2018
Jelgava_Sniega segas biezums, stundas vidējais_01.01.2019-31.12.2019.xls
https://www.meteo.lv/meteorologija-datu-meklesana/? iBy=station&nid=461&pMonitoringType=METEOROLOGY&iStation=30036&iParameter=4341&iDateFrom=01.01.2019&iDateTill=31.12.2019
Jelgava_Sniega segas biezums, stundas vidējais_01.01.2020-31.12.2020.xls
https://www.meteo.lv/meteorologija-datu-meklesana/? iBy=station&nid=461&pMonitoringType=METEOROLOGY&iStation=30036&iParameter=4341&iDateFrom=01.01.2020&iDateTill=31.12.2020
Jelgava_Sniega s

In [123]:
df=[]

for enum, year in enumerate(range(startYear,endYear+1)):
    StartDate="01.01."+str(year)
    EndDate="31.12."+str(year)
    fname=stacijuSaraksts[str(stationID)]+"_" \
    +parametruSaraksts[str(paramID)] + "_" \
    +StartDate+"-" \
    +EndDate+".xls"
    #df[enum]=pd.read_excel(fname)
    df.append(pd.read_excel(fname,skiprows=1,parse_dates=["Datums \ Laiks"],index_col=0,dayfirst=True))
    df[enum]["vidējais sniega segas biezums, cm"]=df[enum].mean(axis=1)
    #df[enum]["vidējā"]=df[enum].mean(axis=1)

In [124]:
sniegaSega=pd.concat(df)
sniegaSega=sniegaSega[["vidējais sniega segas biezums, cm"]]
sniegaSega

,"vidējais sniega segas biezums, mm"
Datums \ Laiks,
2017-01-01,0.000000
2017-01-02,0.166667
2017-01-03,0.291667
2017-01-04,6.750000
2017-01-05,7.375000
...,...
2021-12-27,10.000000
2021-12-28,9.125000
2021-12-29,9.000000


In [80]:
nokrisni.set_index('Datums \ Laiks',inplace=True)

,summa
Datums \ Laiks,
2016-08-25,0.0
2016-08-26,0.0
2016-08-27,0.0
2016-08-28,0.0
2016-08-29,0.1
...,...
2021-12-27,0.0
2021-12-28,0.0
2021-12-29,0.0


In [173]:

kopejais=nokrisni.join(temperatura, how="outer").join(sniegaSega, how="outer").join(relMitrums, how="outer")
kopejais

,summa,"dienas vidējā temperatūra, C","vidējais sniega segas biezums, mm",vidējais relatīvais mitrums
Datums \ Laiks,,,,
2017-01-01,0.2,6.016667,0.000000,94.000000
2017-01-02,0.8,0.800000,0.166667,95.500000
2017-01-03,0.0,-2.641667,0.291667,89.583333
2017-01-04,4.0,-4.291667,6.750000,88.958333
2017-01-05,0.2,-13.229167,7.375000,68.000000
...,...,...,...,...
2021-12-27,0.0,-3.950000,10.000000,89.083333
2021-12-28,0.0,-8.379167,9.125000,90.000000
2021-12-29,0.0,-7.241667,9.000000,72.000000


In [174]:
kopejais


,summa,"dienas vidējā temperatūra, C","vidējais sniega segas biezums, mm",vidējais relatīvais mitrums
Datums \ Laiks,,,,
2017-01-01,0.2,6.016667,0.000000,94.000000
2017-01-02,0.8,0.800000,0.166667,95.500000
2017-01-03,0.0,-2.641667,0.291667,89.583333
2017-01-04,4.0,-4.291667,6.750000,88.958333
2017-01-05,0.2,-13.229167,7.375000,68.000000
...,...,...,...,...
2021-12-27,0.0,-3.950000,10.000000,89.083333
2021-12-28,0.0,-8.379167,9.125000,90.000000
2021-12-29,0.0,-7.241667,9.000000,72.000000


In [143]:
import math

In [144]:
def relMitrums2mitDeficits(t,h):
    return((math.exp((17.3*t)/(t+237.3))*6.11)*(1-(h/100)))

In [175]:
kopejais["mitruma deficīts"]=kopejais.apply(lambda row : relMitrums2mitDeficits(row["dienas vidējā temperatūra, C"], row["vidējais relatīvais mitrums"]), axis=1 )

In [176]:
kopejais

,summa,"dienas vidējā temperatūra, C","vidējais sniega segas biezums, mm",vidējais relatīvais mitrums,mitruma deficīts
Datums \ Laiks,,,,,
2017-01-01,0.2,6.016667,0.000000,94.000000,0.562314
2017-01-02,0.8,0.800000,0.166667,95.500000,0.291406
2017-01-03,0.0,-2.641667,0.291667,89.583333,0.523828
2017-01-04,4.0,-4.291667,6.750000,88.958333,0.490560
2017-01-05,0.2,-13.229167,7.375000,68.000000,0.704053
...,...,...,...,...,...
2021-12-27,0.0,-3.950000,10.000000,89.083333,0.497681
2021-12-28,0.0,-8.379167,9.125000,90.000000,0.324364
2021-12-29,0.0,-7.241667,9.000000,72.000000,0.992429


In [177]:
kopejais["SSL"]=0
kopejais["SSS"]=0
kopejais

,summa,"dienas vidējā temperatūra, C","vidējais sniega segas biezums, mm",vidējais relatīvais mitrums,mitruma deficīts,SSL,SSS
Datums \ Laiks,,,,,,,
2017-01-01,0.2,6.016667,0.000000,94.000000,0.562314,0,0
2017-01-02,0.8,0.800000,0.166667,95.500000,0.291406,0,0
2017-01-03,0.0,-2.641667,0.291667,89.583333,0.523828,0,0
2017-01-04,4.0,-4.291667,6.750000,88.958333,0.490560,0,0
2017-01-05,0.2,-13.229167,7.375000,68.000000,0.704053,0,0
...,...,...,...,...,...,...,...
2021-12-27,0.0,-3.950000,10.000000,89.083333,0.497681,0,0
2021-12-28,0.0,-8.379167,9.125000,90.000000,0.324364,0,0
2021-12-29,0.0,-7.241667,9.000000,72.000000,0.992429,0,0


In [178]:
kopejais.columns

Index(['summa', 'dienas vidējā temperatūra, C',
       'vidējais sniega segas biezums, mm', 'vidējais relatīvais mitrums',
       'mitruma deficīts', 'SSL', 'SSS'],
      dtype='object')

In [181]:
kopejais.to_excel("darbaFails.xls")

ModuleNotFoundError: No module named 'keras'